In [ ]:
'''
conda environment setup (may not work with other python / arcgis versions):

conda create -n arcgisDev2 python=3.7.11
conda activate arcgisDev2
conda install -c esri arcgis=1.9.1
conda install jupyter
conda install pywin32
'''

### CONFIGURATION USERNAMES AND PASSWORDS
import os, sys
admin_username = os.environ["AGOL_USERNAME"]
admin_password = os.environ["AGOL_PASSWORD"]

from arcgis.gis import *
import json
gis = GIS(username=admin_username, password=admin_password)

workspace =  os.path.dirname(os.path.abspath("__file__"))
input_file_name = workspace + r'\web_mapping_application_data.json'
agol_item_title = 'DEV_WEBAPP_PYTHONTEST'

# delete the AGOL item if it already exists
search_results = gis.content.search(query=agol_item_title)
for result in search_results:
    if result['title'] == agol_item_title:
        print('deleting: title = {} itemid = {}\n'.format(result['title'], result.itemid))
        result.delete()

with open(input_file_name) as input_file:
    print('reading {}\n'.format(input_file_name))
    json_input = input_file.read()
# default_obj = json.loads(json_input)
    
ip = {
    "title": agol_item_title,
    "type": "Web Mapping Application",
    "typeKeywords": [
        "JavaScript",
        "Map",
        "Mapping Site",
        "Online Map",
        "Ready To Use",
        "WAB2D",
        "Web AppBuilder",
        "Web Map",
        "Registered App"
    ]
}

print('input_file_name: {}\n'.format(input_file_name))
# print('default_obj: {}'.format(default_obj))
print('ip: {}\n'.format(ip))

# item = gis.content.add(item_properties=ip, folder="DevTest", data=input_file_name) # works but no data (in arcgis 1.9.1 as well)
# item = gis.content.add(item_properties=ip, folder="DevTest", data=default_obj) # error in arcgis 1.9.1
# create the item
item = gis.content.add(item_properties=ip, folder="DevTest")

# set the url for the web app
ip['url'] = "https://your.agol.url.com/apps/webappviewer/index.html?id={}".format(item.itemid)


# attempt to set the thumbnail url
rasp_web_app_thumbnail_item_title = 'WEB_APP_THUMBNAIL_RASP'
rasp_web_app_thumbnail_item = None
rasp_web_app_thumbnail_url = None
search_results = gis.content.search(query=rasp_web_app_thumbnail_item_title)
for result in search_results:
    if result['title'] == rasp_web_app_thumbnail_item_title:
        rasp_web_app_thumbnail_item = result
if rasp_web_app_thumbnail_item:
    rasp_web_app_thumbnail_url = 'https://your.agol.url.com/sharing/rest/content/items/{}/data'.format(rasp_web_app_thumbnail_item.itemid)
    print('thumbnail url set to: {}'.format(rasp_web_app_thumbnail_url))
    
if rasp_web_app_thumbnail_url:
    item.update(item_properties=ip, thumbnail=rasp_web_app_thumbnail_url)
else:
    item.update(item_properties=ip)

# item.update(item_properties=ip)

# print item description
for key in item:
    print("{} = {}".format(key, item[key]))

print('item:')
print(item)
print('item.get_data():')
print(item.get_data(try_json=True)) # no data in AGOL assistant


# find icon urls to use, these are public images hosted in AGOL
icon_data = {
    'GENERATE_WAYLINES_ICON' : {},
    'RBL_TO_APP_BLOCK_ICON' : {},
    'DELETE_UNASSOCIATED_APPLICATION_BLOCKS_ICON' : {},
    'COPY_PASTE_ICON' : {},
    'DELETE_WAYLINES_ICON' : {}
}

for icon_name in icon_data:
    search_results = gis.content.search(query=icon_name)
    
    for result in search_results:
        if result['title'] == icon_name:
            icon_data[icon_name]['itemid'] = result.itemid
            icon_data[icon_name]['url'] = 'https://your.agol.url.com/sharing/rest/content/items/{}/data'.format(result.itemid)
            
            
print('\nicon_data:')
print(icon_data)
print('')
            
default_obj = json.loads(json_input)

default_obj['appItemId'] = item.itemid # set item id

# mapping from labels to icon name
gp_icon_names = {
    'Copy Paste' : 'COPY_PASTE_ICON',
    'RBL to APP BLK' : 'RBL_TO_APP_BLOCK_ICON',
    'Delete Unassociated App Blocks' : 'DELETE_UNASSOCIATED_APPLICATION_BLOCKS_ICON'
}

# location for custom widgets
widget_sets = [
    'widgetOnScreen',
    'widgetPool'
]

# go through the widgets and set the url for every custom widget found
for widget_set in widget_sets:
    widgets = default_obj.get(widget_set)['widgets']
    for w in widgets:
        if w.get('label') in gp_icon_names:
            # set url for icon
            icon_name = gp_icon_names[w.get('label')]
            print('icon_name: {}'.format(icon_name))
            w['icon'] = icon_data[icon_name]['url']

item.update(data=default_obj)
print('item:')
print(item)
print('item.get_data():')
print(item.get_data(try_json=True)) # data JSON seen in AGOL assistant, can't open the app

widgets = item.get_data(try_json=True).get('widgetOnScreen')['widgets']
for w in widgets:
    if w.get('label') == 'Generate Waylines':
        print(w)